# 그래프 DB 구축 실습
- MS의 GraphRAG 패키지를 주로 활용하여 진행합니다.

- 작업 디렉터리 설정
- 파일 시스템 작업을 효율적으로 관리하기 위해 pathlib 라이브러리를 사용하여 GraphRAG에서 사용할 작업 디렉터리를 생성합니다.
- pathlib 라이브러리를 활용하면 파일 시스템 경로를 객체지향적으로 다룰 수 있습니다.

In [1]:
from pathlib import Path

working_dir = Path('working_directory')
working_dir.mkdir(parents=True, exist_ok=True)

- 작업 디렉터리를 생성한 후에는 GraphRAG 초기화를 수행합니다.

In [2]:
!graphrag init --root ./working_directory

⠋ GraphRAG Indexer 
Initializing project at C:\workspace\python\rag_master\ch05\working_directory
⠋ GraphRAG Indexer 


- 이 명령어를 실행하면 두 개의 설정 파일이 생성됩니다.
- .env: 오픈AI API키 등 환경 변수 저장
- settings.yaml: 전체 파이프라인의 설정 저장

- .env 파일 입력
```
GRAPHRAG_API_KEY=OpenAPI API키
```

- settings.yaml 파일 입력
- GraphRAG의 동작과 관련된 전체 파라미터를 담은 설정 파일입니다.
- 이 파일을 통해 데이터 처리, LLM 설정, 병렬 처리등 다양한 요소를 세부적으로 설정합니다.

- llm 파라미터
```yaml
models:
    api_key: ${GRAPHRAG_API_KEY}
    type: openai_chat
    model: gpt-4o-mini
```

- 문서 분할 설정: chunks 파라미터
    - 대용량 텍스트를 처리하기 위해 문서를 청크로 분할하는 설정입니다.
    - group_by_columns는 지정된 칼럼별로 문서를 그룹화하여 청크를 생성하는 옵션입니다.
```yaml
chunks:
    size: 300
    overlap: 100
    group_by_columns: [id]
```

- 엔티티 추출 설정: extract_graph 파라미터
    - 지식 그래프 구축 과정에서 엔티티를 추출하는 단계를 위한 설정입니다.
    - prompt에는 LLM에 전달할 텍스트 파일의 위치를 지정합니다.
    - entity_types는 추출 대상의 엔티티 종류를 지정합니다.
    - 마지막으로 max_gleanings는 반복 추출 횟수를 지정합니다.
```yaml
extract_graph:
    prompt: "prompts/extract_graph.txt"
    entity_types: [organization, person, geo, event]
    max_gleanings: 1
```

- 요약설정: summarize_descriptions
    - 같은 엔티티에 대한 설명이 여러개 일때, 이들을 하나로 통합하는 단계의 설정값입니다.
    - 생성되는 요약문의 최대 길이는 max_length 파라미터를 통해 제한될 수 있습니다.
```yaml
summarize_descriptions:
    model_id: default_chat_model
    prompt: "prompts/summarize_descriptions.txt"
    max_length: 500
```

- 주장 추출 설정: claim_extraction 파라미터
    - 텍스트에서 특정 엔티티와 관련된 주장들을 추출하고, 이를 체계적으로 정리하는 방식을 지정하는 단계의 설정값입니다.
    - 각 주장에 대해 주체, 대상, 주장 유형, 상태, 날짜 등의 정보를 구조화된 형태로 추출합니다.
```yaml
extract_claims:
    enable: false
    model_id: default_chat_model
    prompt: "prompts/extract_claims.txt"
    description: "Any claims or facts that could be relevant to information discovery."
    max_gleanings: 1
```

- 커뮤니티 요약문 설정: community_reports 파라미터
    - 특정 커뮤니티의 주요 엔티티들과 그들 간의 관계를 분석하여 종합적인 보고서를 생성하는 단계
    - 이 과정을 완료하면 그래프 RAG를 실행할 기본 환경 구성이 끝납니다. 
```yaml
community_reports:
    model_id: default_chat_model
    graph_prompt: "prompts/community_report_graph.txt"
    text_prompt: "prompts/community_report_text.txt"
    max_length: 2000
    max_input_length: 8000
```

## 지식 그래프 구축
- working_directory 폴더 내에 input 폴더를 만들고, 그래프 DB로 사용할 문서를 업로드 합니다.

In [8]:
# 워킹 디렉토리 안에 input 폴더 생성
input_dir = working_dir / 'input'
input_dir.mkdir(parents=True, exist_ok=True)

- 그래프 구축에 사용할 글은 투자에 관한 서적으로, How to Invest Money라는 책의 텍스트 파일을 사용합니다.

In [9]:
import shutil
# data_file을 input 폴더로 복사

source_file = './data/How_to_invest_money.txt'

shutil.copy(source_file, input_dir)

'working_directory\\input\\How_to_invest_money.txt'

- 다음 명령어를 입력하여 그래프 DB 구축을 실행합니다.

In [10]:
!graphrag index --root ./working_directory


⠋ GraphRAG Indexer 
Logging enabled at 
C:\workspace\python\rag_master\ch05\working_directory\logs\indexing-engine.log
⠋ GraphRAG Indexer 
⠹ GraphRAG Indexer 
⠴ GraphRAG Indexer 
⠧ GraphRAG Indexer 
🚀 LLM Config Params Validated
⠏ GraphRAG Indexer 
⠋ GraphRAG Indexer 
⠸ GraphRAG Indexer 
⠴ GraphRAG Indexer 
⠇ GraphRAG Indexer 
🚀 Embedding LLM Config Params Validated
⠋ GraphRAG Indexer 
⠋ GraphRAG Indexer 
Running standard indexing.
⠙ GraphRAG Indexer 
└── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) - 100%  0…
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) - 100%  0…
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) - 100%  0…
🚀 create_base_text_units
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) - 100%  0…
├── create_base_text_units ------------------------------- 100% 0:00:00 0:00:00
                                                    id  ..

[2025-05-24T10:21:48Z WARN  lance::dataset::write::insert] No existing dataset at C:\workspace\python\rag_master\ch05\working_directory\output\lancedb\default-entity-description.lance, it will be created
[2025-05-24T10:21:49Z WARN  lance::dataset::write::insert] No existing dataset at C:\workspace\python\rag_master\ch05\working_directory\output\lancedb\default-community-full_content.lance, it will be created
[2025-05-24T10:21:51Z WARN  lance::dataset::write::insert] No existing dataset at C:\workspace\python\rag_master\ch05\working_directory\output\lancedb\default-text_unit-text.lance, it will be created


- 그래프 DB 구축은 매우 복잡한 워크플로우를 거쳐 진행됩니다. 전체 과정을 크게 2가지 범주로 나눌수 있습니다.

1. Base processing 단계
    - 원본 문서를 처리하기 위한 기초적인 작업을 수행합니다.
    - 문서 분할을 수행하고, 지식 그래프 및 그래프 증강의 초기 버전을 구현하는 단계입니다.
    - 문서 분할(create_base_text_units)을 통해 긴 문서를 여러 개의 텍스트 단위로 쪼갠 뒤, 이를 바탕으로 엔티티 추출, 기본 그래프 생성, 커뮤니티 탐지, 임베딩 작업(create_base_extracted_entities, create_base_entity_graph), 공변량 추출(create_final_covariates)을 진행합니다.
    - 이 과정을 통해 초기 형태의 엔티티 그래프를 구축하고, 각 엔티티 및 관계들을 식별할 수 있는 토대를 마련합니다.
2. Enrichment(고도화) 단계
    - 구축된 그래프에 대해 추가적인 보강 작업을 수행하는 단계입ㄴ디ㅏ.
    - 구축된 지식 그래프를 보강하고, 커뮤니티 요약 및 시각화를 수행하여 더욱 풍부한 정보를 제공할 수 있도록 완성도를 높이는 과정입니다.
    - 엔티티, 관계, 커뮤니티 데이터의 정제(create_final_entities, create_final_communities, create_final_relationships), 그래프 시각화(create_final_nodes) 등을 거쳐 더욱 풍부하고 의미있는 그래프 DB를 형성합니다.
    - 커뮤니티 요약(create_final_community_reports) 생성을 통해 특정 커뮤니티를 중심으로 요약된 정보와 통계, 임베딩 등을 도출합니다.

### 1. 문서 분할: create_base_text_units
- 입력 텍스트를 여러 청크로 분할하는 작업을 수행합니다.
- LLM 모델의 입력 길이 제한을 고려하여 긴 문서를 여러 부분으로 나누고, 병렬 처리를 가능하게 하며 메모리 효율성을 높입니다.
- 토큰 수 기준 분할로, 분할 기준이 되는 토큰 수는 앞선 settings.yaml 설정값을 따릅니다.

```text
원문: "The Project Gutenberg eBook of How to Invest Money..."
결과:
- 청크 1: "The Project Gutenberg eBook..." (300토큰) 
- 청크 2: "...eBook of How to Invest Money..." (300토큰, 이전 100토큰 포함)
```

### 엔티티 추출: create_base_extracted_entities
- 텍스트 조각에서 중요한 엔티티를 추출하고, 이를 기반으로 기본 엔티티 그래프를 생성합니다.
- 텍스트 내에서 중요한 엔티티를 식별하고, 그들 간의 관계를 파악하기 위한 초기작업이라고 생각하면 됩니다.

```text
입력: "Project Gutenberg is a digital library..."
출력:
- 엔티티: PROJECT GUTENBERG
- 유형: ORGANIZATION
- 설명: "무료 전자책을 제공하는 디지털 도서관"
```

### 3. 기본 엔티티 그래프 처리 및 확장: create_base_entity_graph
- 엔티티 그래프를 바탕으로 추가적인 그래프 처리 및 분석을 수행하는 단계입니다.
- 개별 텍스트 유닛에서 생성된 엔티티 그래프를 통합하고, 커뮤니티 탐지 그래프 임베딩 생성등의 작업을 수행하여 그래프를 확장하고 정제합니다.
1. 그래프 통합: 개별 텍스트 유닛에서 생성된 그래프를 하나의 통합된 그래프로 병합하여 전체적인 데이터 구조를 구성합니다.
2. 커뮤니티 탐지: 그래프 클러스터링 알고리즘을 적용하여 유사한 엔티티들을 그룹화하고, 커뮤니티 구조를 파악합니다.
3. 그래프 임베딩 생성: 그래프의 구조적 특성을 벡터화하여 노드 임베딩을 생성합니다.

```json
{
    'id':'GEORGE GARR HENRY',
    'data': {
        'type': 'PERSON',
        'description': 'George Garr Henry was Vice-President of Guaranty ...',
        ;cluster': '5',
        'level': '1',
        'degree': '10'
    }
}
```
- 커뮤니티 탐지: cluster번호'5'를 부여받았으며, 이는 5번 커뮤니티에 속한 다른 엔티티들과 유사하거나 연관성이 있음을 나타냅니다. level은 '1'로 커뮤니티 계층 구조에서의 위치를 보여줍니다.
- 그래프 속성: degree가 '10'으로 표시되어 있는데 이 엔티티가 그래프 내에서 다른 엔티티와 10개의 연결을 가지고 있음을 의미합니다. 이를 통해 해당 엔티티의 중요도나 영향력을 유추할 수 있습니다.


### 4. 공변량 추출: create_final_covariates
- 앞선 단계들에서 얻은 텍스트 유닛과 추출된 엔티티들을 사용하여 공변량을 추출합니다.
- 공변량은 통계나 머신러닝 모델에서 종속 변수에 영향을 미칠 수 있는 독립 변수를 의미하며, 여기서는 텍스트에서 주장을 추출하여 공변량으로 사용합니다.
- 그래프 DB 구축에 사용할 수 있는 정보를 추가로 추출하여 더 풍부한 정보를 담은 DB를 구축하는 데 목적이 있습니다.


### 5. 최종 엔티티 데이터 생성: create_final_entities
- 앞선 단계에서 얻은 기본 엔티티 그래프의 노드 데이터를 기반으로 최종적인 엔티티 데이터를 생성하는 단계입니다.
1. 엔티티 데이터 정제: 엔티티 데이터의 중복 제거, 빈 값 필터링 등을 통한 정제를 수행합니다.
2. 임베딩 생성: 엔티티의 이름과 설명에 대한 임베딩을 생성하여 의미를 벡터화합니다. 그래프 임베딩이 그래프의 구조를 임베딩한 것이라면 이번 임베딩은 엔티티 텍스트를 임베딩한 것입니다
3. 데이터 구조 개선: 칼럼명 변경 및 데이터 정리를 통해 이후 분석에 용이하도록 데이터 구조를 개선합니다.

### 6. 최종 노드 데이터 생성 및 그래프 레이아웃 계산: create_final_nodes
- 엔티티 추출작업이 완료되어 엔티티들을 시각화하기 위한 작업을 진행합니다.
- 그래프의 레이아웃을 계산하여 노드의 위치 정보를 생성하고, 최종 노드 테이블을 완성합니다.
- 이때 노드의 위치 정보 생성에는 차원 축소 기법이 사용됩니다.

#### 차원 축소 기법을 사용한 노드 좌표 생성
- 차원 축소 기법을 활용하면 노드의 임베딩을 바탕으로 좌표 정보를 생성할 수 있습니다.
1. 임베딩 벡터 준비: 각 노드에 대한 그래프 임베딩 벡터를 수집합니다.
2. 차원 축소 수행: 
    - 임베딩 벡터를 사용하여 고차원 공간에서 2D 또는 3D 공간으로 차원 축소를 수행합니다.
    - UMAP(Uniform Manifold Approximation and Projection)과 같은 차원 축소 기법을 적용합니다.
    - UMAP은 고차원 데이터의 구조를 보존하면서 2D 공간으로 변환하는 알고리즘입니다.
    - 가까운 노드들은 2D 공간에서도 가깝게, 먼 노드들은 멀게 배치합니다.
    - 이 과정에서 노드간의 유사성 관계가 최대한 보존되어 원래 그래프의 구조적 특성을 직관적으로 시각화할 수 있게 됩니다.
3. 좌표 할당: 축소된 벡터 값을 노드의 x,y 좌표(필요에 따라 z좌표도 포함)로 할당합니다.

```json
{
    'title': 'PROJECT GUTENBERG',
    'type': 'ORGANIZATION',
    'description': 'Project Gutenberg is ...',
    'community': '10',
    'degree': 23,
    'size': 23,
    'graph_embedding: [
        -0.04243285953998566,
        ...
    ],
    'x': 16.86814308166504,
    'y': 8.587206840515137,
}
```

- title: 노드의 제목을 나타냅니다.
- type: 노드의 유형을 나타냅니다.
- description: 노드에 대한 상세 설명입니다.
- community: 노드가 속한 커뮤니티를 나타내는 번호입니다.
- degree: 노드의 연결 정도를 나타냅니다.
- size: 노드의 크기를 나타냅니다.
- graph_embedding: 노드의 의미를 벡터화한 임베딩 값들의 리스트입니다.
- x, y: 그래프 시각화에서 노드의 위치를 나타내는 좌표값입니다.

### 7 커뮤니티 요약문 생성: create_final_community_reports
- 커뮤니티 요약문을 생성하는 단계입니다.
- 앞서 만든 각 커뮤니티에 대한 요약문을 생성하고 임베딩하여 최종 커뮤니티 요약문 테이블을 만듭니다.

```json
{
    'community': '0',
    'level': '0',
    'rank': 7.5,
    'title': 'Traction System and Urban Growth in Ameracan Cities',
    'rank_explanation': 'The impact severity rating is high due to the cretical role ...',
    'summary' : 'The community is centered around traction systems in major American...',
    'findings': [
        {
            'explanation': 'Traction system in New York Chicago are ...'
            'summary': 'Financial challenges of traction systems'
        },
        {
            'explanation': 'The rapid growth of population and...'
            'summary': 'Impact of urban growth on traction systems'
        }
    ]
}
```

- level: 요약문이 다루는 커뮤니티 계층 레벨입니다.
- rank: 요약문의 중요도 점수입니다.
- title: 요약문의 제목입니다.
- rank_explanation: 중요도 점수에 대한 설명입니다. 도시 인프라에서 교통 시스템의 중요성과 그들이 직면한 재정적, 법적 과제들을 근거로 높은 점수가 부여되었음을 설명합니다.
- summary: 요약문의 주요 내용들입니다.
- findings: 주요 발견 사항들의 목록입니다. 각 발견사항은 'explanation'(상세설명)과 'summary'(요약)로 구성되어 있습니다.